<a href="https://colab.research.google.com/github/ardeeshany/Deep-Learning-Projects/blob/main/2022_05_Pytorch_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load required packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [17]:
# Load and PreProcess Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])])
trainset = torchvision.datasets.CIFAR10(root = "./", train = True, download = True, transform = transform) # 50K obs
testset = torchvision.datasets.CIFAR10(root = "./", train = False, download = True, transform = transform) # 10K obs

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 128, shuffle = True, num_workers = 2) # 32x32 images
test_loader = torch.utils.data.DataLoader(testset, batch_size = 128, shuffle = True, num_workers = 2)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
from torch.nn.modules.batchnorm import BatchNorm2d
# Building block of Network
def conv_block(in_channels, out_channels, activation = 'relu', *args, **kwargs):
    activations = nn.ModuleDict([
                ['relu', nn.ReLU()],
                ['lkrelu', nn.LeakyReLU()],
    ])

    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, *args, **kwargs),
        nn.BatchNorm2d(out_channels),
        activations[activation]
    )


# Building the Network
class MyClassifier(nn.Module):

    def __init__(self, num_channels, out_classes):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_channels = num_channels, out_channels = 32, activation = 'relu', kernel_size = 3, padding = 1, stride = 1),
            conv_block(in_channels = 32, out_channels = 64, activation = 'relu', kernel_size = 3, padding = 1, stride = 1),
        )

        self.decoder = nn.Sequential(
            nn.Linear(64*32*32, 128),
            nn.Sigmoid(),
            nn.Linear(128, out_classes)
        )

    def forward(self, x):
        x = self.encoder(x)

        x = x.view(-1, 64*32*32)

        x = self.decoder(x)
       
        return x




In [19]:
# Compile the model
net = MyClassifier(num_channels=3, out_classes=10)
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = net.parameters(), lr = 0.01)

In [20]:
# Train the model
for epoch in range(1):
    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = criteria(outputs, labels)

        loss.backward()

        optimizer.step()
        print("Step " + str(i) + " is completed!")


Step 0 is completed!
Step 1 is completed!
Step 2 is completed!
Step 3 is completed!
Step 4 is completed!
Step 5 is completed!
Step 6 is completed!
Step 7 is completed!
Step 8 is completed!
Step 9 is completed!
Step 10 is completed!
Step 11 is completed!
Step 12 is completed!
Step 13 is completed!
Step 14 is completed!
Step 15 is completed!
Step 16 is completed!
Step 17 is completed!
Step 18 is completed!
Step 19 is completed!
Step 20 is completed!
Step 21 is completed!
Step 22 is completed!
Step 23 is completed!
Step 24 is completed!
Step 25 is completed!
Step 26 is completed!
Step 27 is completed!
Step 28 is completed!
Step 29 is completed!
Step 30 is completed!
Step 31 is completed!
Step 32 is completed!
Step 33 is completed!
Step 34 is completed!
Step 35 is completed!
Step 36 is completed!
Step 37 is completed!
Step 38 is completed!
Step 39 is completed!
Step 40 is completed!
Step 41 is completed!
Step 42 is completed!
Step 43 is completed!
Step 44 is completed!
Step 45 is completed

In [21]:
# Validate
total , correct = 0, 0

net.eval()

for i, data in enumerate(test_loader):
    inputs, labels = data

    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, dim = 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print("The accuracy is: " + str(correct/total*100)+"%")    



The accuracy is: 23.630000000000003%
